In [1]:
import psycopg2
import pandas as pd

In [2]:
# Let's work with the first 250 rows of data to save time.
df = pd.read_csv("Web Scraping/IMDB top 1000 (cleaned).csv").rename(columns = {"Unnamed: 0":"Movie_ID"}).head(250)
df.head(2)

,Movie_ID,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Year,Directors,Stars,Votes,Gross($M)
0,0,Esaretin Bedeli,13+,142,Drama,9.3,80,Two imprisoned men bond over a number of years...,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2377444,28.34
1,1,Baba,16,175,"Crime, Drama",9.2,100,An organized crime dynasty's aging patriarch t...,1972,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1646788,134.97


In [3]:
df.shape

(250, 13)

### Create `imdb` table in DB

In [ ]:
conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
cur = conn.cursor()
command = (
        """
        CREATE TABLE imdb (
            Movie_ID INTEGER PRIMARY KEY,
            Title VARCHAR(255) NOT NULL,
            Certificate VARCHAR(255) NOT NULL,
            Duration INTEGER NOT NULL,
            Genre VARCHAR(255) NOT NULL,
            Rate FLOAT NOT NULL,
            Metascore INTEGER,
            Description VARCHAR(4096) NOT NULL,
            Year INTEGER NOT NULL,
            Directors VARCHAR(255) NOT NULL,
            Stars VARCHAR(255) NOT NULL,
            Votes INTEGER,
            Gross FLOAT
        )
        """)
cur.execute(command)
cur.close()
conn.commit()
conn.close()

### Insert dataframe into DB

In [16]:
conn = psycopg2.connect('dbname=imdb user=postgres password=1234')
cur = conn.cursor()

records = df.to_dict('record')
sql = 'INSERT INTO imdb VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

for k,v in enumerate(records):
    cur.execute(sql, (k, v['Title'], v['Certificate'], v['Duration'], v['Genre'], v['Rate'], v['Metascore'], v['Description'], v['Year'], v['Directors'], v['Stars'], v['Votes'], v['Gross($M)']))

cur.close()
conn.commit()
conn.close()

### Export data from DB

In [17]:
def find_popular(series):
    col = series.name
    df= pd.DataFrame(series)
    df[col] = df[col].str.split(',').apply(lambda x: [i.strip() for i in x])
    for index, row in df.iterrows():
        for column in row[col]:
            df.at[index, column] = 1
    df = df.fillna(0)
    return df.iloc[:,1:].sum().sort_values(ascending=False)[:1]
    

In [18]:
def get_database():
    global top_10_profitable_movies, before_2000, after_2000, num_of_movies, highest_rated_movie, popular_actor, popular_director, df_all
    
    # --Number of Movies
    query_1 = f"select count(Title) from imdb;"

    # --Highest Rated Movie 
    query_2 = f"select Title, Rate from imdb order by Rate Desc limit 1;"

    # --Top 10 Most Profitable Movies
    query_3 = f"select Title, Gross from imdb order by Gross desc limit 10;"

    # --Number of Movies before 2000
    query_4 =f"select count(year) from imdb where year<2000;"

    # --Number of Movies after 2000
    query_5 =f"select count(year) from imdb where year>=2000;"

    # most_popular_star & most_popular_director
    query_6 =f"select * from imdb;"

    conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
    cur = conn.cursor()

    cur.execute(query_1)
    num_of_movies = cur.fetchall()[0][0]

    cur.execute(query_2)
    highest_rated = cur.fetchall()[0]
    highest_rated_movie = f"{highest_rated[0]}    ({highest_rated[1]} Rated)"

    cur.execute(query_3)
    the_data = cur.fetchall()
    top_10_profitable_movies = pd.DataFrame(the_data, columns=[desc[0] for desc in cur.description])

    cur.execute(query_4)
    before_2000= cur.fetchall()[0][0]

    cur.execute(query_5)
    after_2000= cur.fetchall()[0][0]

    cur.execute(query_6)
    the_data = cur.fetchall()
    df_all = pd.DataFrame(the_data, columns=[desc[0] for desc in cur.description])
    popular_a= find_popular(df_all['stars'])
    popular_actor = f"{popular_a.index[0]}    ({int(popular_a.values[0])} Movies)"
    popular_d= find_popular(df_all['directors'])
    popular_director = f"{popular_d.index[0]}    ({int(popular_d.values[0])} Movies)"

    cur.close()
    conn.close()

### Create a Dashboard

In [20]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Output, Input, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go

import pandas as pd
import psycopg2
import pandas.io.sql as sqlio

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

top_10_profitable_movies = pd.DataFrame()
before_2000 = 0
after_2000 = 0
num_of_movies = ""
highest_rated_movie = ""
popular_actor = ""
popular_director = ""
df_all = pd.DataFrame()
fig=''
fig2=''
get_database()

def get_figure():
    global top_10_profitable_movies, before_2000, after_2000, num_of_movies, highest_rated_movie, popular_actor, popular_director, df_all, fig, fig2
    # Use the hovertext kw argument for hover text
    fig = go.Figure(data=[go.Bar(x= top_10_profitable_movies['title'].values, 
                                 y= top_10_profitable_movies['gross'].values)])

    # Customize aspect
    fig.update_traces(marker_color='rgb(97, 185, 231)', 
                      marker_line_color='rgb(97, 185, 231)')

    fig.update_layout(title_text='Top 10 Movies by Gross',
                      height= 300,
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor ='rgba(0,0,0,0)',
                      bargap=0.45,
                      font_color="white",
                      yaxis = dict(
                            tickfont_size=12,
                        ),
                      xaxis = dict(
                            tickfont_size=7,
                            tickangle=40,
                        ),
                      margin=dict(l=50, r=0, b=50, t=100)
                      )

    fig.update_yaxes(showgrid=True, 
                     gridwidth=1, 
                     gridcolor='DarkBlue')

    # Use `hole` to create a donut-like pie chart
    fig2 = go.Figure(data=[go.Pie(labels=['Before 2000', 'After 2000'], 
                                  values=[before_2000, after_2000], 
                                  hole=.8)])

    fig2.update_traces(marker=dict(colors=['rgb(221, 81, 121)',
                                           'rgb(114, 221, 193)']))

    fig2.update_layout(title_text='The Ratio of the Movies before/after 2000',
                      height= 350,
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor ='rgba(0,0,0,0)',
                      font_color="white",
                      margin=dict(l=0, r=0, b=100, t=100)
                      )

get_figure()

app.layout = html.Div([
                dbc.Row(
                    dbc.Col([
                            ################################ 1st Part #################################
                        dbc.Row(
                            dbc.Col([
                                dbc.NavbarSimple(
                                    children=[
                                        dbc.NavItem(dbc.NavLink("ACTION", href="/action")),
                                        dbc.NavItem(dbc.NavLink("COMEDIES", href="/comedies")),
                                        dbc.NavItem(dbc.NavLink("ROMANTIC", href="/romantic")),
                                        dbc.NavItem(dbc.NavLink("ADVENTURE", href="/adventure")),
                                        dbc.NavItem(dbc.NavLink("SCI-FI", href="/sci-fi")),
                                        dbc.NavItem(dbc.NavLink("DRAMA", href="/drama")),
                                        dbc.NavItem(dbc.NavLink("HORROR", href="/horror")),
                                        dbc.NavItem(dbc.NavLink("HISTORICAL", href="/historical")),
                                        dbc.NavItem(dbc.NavLink("OTHERS", href="/others")),
                                    ],
                                    brand="IMDB",
                                    brand_href="/home",
                                    id='nav',
                                ),
                                html.P("Fatih's Dashboard", id='name'),
                            ]),
                        ),
                        ################################ 2nd Part #################################
                        dbc.Row([
                            dbc.Col([
                                
                                dbc.Row([
                                    dbc.Col([
                                        html.P('Number of Movies'),
                                        html.H5(id='num_of_movies')
                                    ], width=5),
                                    dbc.Col([
                                        html.P('Highest Rated Movie'),
                                        html.H5(id='highest_rated_movie')
                                    ], width=5)
                                ], id='first', justify='center'),
                                
                                dbc.Row([
                                    dbc.Col([
                                        html.P('Popular Actor'),
                                        html.H5(id='popular_actor')
                                    ], width=5),
                                    dbc.Col([
                                        html.P('Popular Director'),
                                        html.H5(id='popular_director')
                                    ], width=5)
                                ],id='second', justify='center')
                                
                            ],id='first-middle', width=4
                            ),
                            
                            dbc.Col(
                                html.Div(id='graph1'),width=4, id='second-middle'
                            ),
                            
                            dbc.Col(
                                html.Div(id='graph2'), width=4, id='third-middle')
                            
                        ],id='graphs'
                        ),

                        ################################ 3rd Part #################################
                        dbc.Row([
                            dbc.Col(dbc.Input(id="input-1", placeholder="id", type="number")),
                            dbc.Col(dbc.Input(id="input-2", placeholder="Title", type="text")),
                            dbc.Col(dbc.Input(id="input-3", placeholder="Certificate", type="text")),
                            dbc.Col(dbc.Input(id="input-4", placeholder="Duration", type="number")),
                            dbc.Col(dbc.Input(id="input-5", placeholder="Genre", type="text")),
                            dbc.Col(dbc.Input(id="input-6", placeholder="Rate", type="number")),
                            dbc.Col(dbc.Input(id="input-7", placeholder="Metascore", type="number")),
                            dbc.Col(dbc.Input(id="input-8", placeholder="Description", type="text")),
                            dbc.Col(dbc.Input(id="input-9", placeholder="Year", type="number")),
                            dbc.Col(dbc.Input(id="input-10", placeholder="Directors", type="text")),
                            dbc.Col(dbc.Input(id="input-11", placeholder="Stars", type="text")),
                            dbc.Col(dbc.Input(id="input-12", placeholder="Votes", type="number")),
                            dbc.Col(dbc.Input(id="input-13", placeholder="Gross", type="number")),
                            ], className="mt-3", justify='center'
                        ),

                        dbc.Row(
                            dbc.Col(html.Div(id='table'), width=12
                            ), className="mt-3", justify='center'
                        ),
                        
                        dbc.Row(
                            dbc.Col([
                                dbc.Button("Show", id='show-1', n_clicks=0),
                                dbc.Button("Add", id='add-1', n_clicks=0),
                                dbc.Button("Update", id='update-1', n_clicks=0),
                                dbc.Button("Delete", id='delete-1', n_clicks=0),
                            ], width=4), className="mt-3", justify='center'
                        ),

                    ]), id='middle'
                ),
            ], id='main-page')

## Callback fonksiyonu
@app.callback(
    [Output(component_id='num_of_movies', component_property='children'),
     Output(component_id='highest_rated_movie', component_property='children'),
     Output(component_id='popular_actor', component_property='children'),
     Output(component_id='popular_director', component_property='children'),
     Output(component_id='graph1', component_property='children'),
     Output(component_id='graph2', component_property='children'),
     Output(component_id='table', component_property='children'),],
    [Input(component_id='show-1', component_property='n_clicks'),
    Input(component_id='add-1', component_property='n_clicks'),
    Input(component_id='delete-1', component_property='n_clicks'),
    Input(component_id='update-1', component_property='n_clicks')],
    [State(component_id='input-1', component_property='value'),
     State(component_id='input-2', component_property='value'),
     State(component_id='input-3', component_property='value'),
     State(component_id='input-4', component_property='value'),
     State(component_id='input-5', component_property='value'),
     State(component_id='input-6', component_property='value'),
     State(component_id='input-7', component_property='value'),
     State(component_id='input-8', component_property='value'),
     State(component_id='input-9', component_property='value'),
     State(component_id='input-10', component_property='value'),
     State(component_id='input-11', component_property='value'),
     State(component_id='input-12', component_property='value'),
     State(component_id='input-13', component_property='value'),
     State(component_id='num_of_movies', component_property='movie'),
     State(component_id='highest_rated_movie', component_property='highest_rate'),
     State(component_id='popular_actor', component_property='star'),
     State(component_id='popular_director', component_property='dirctr'),
     State(component_id='graph1', component_property='chart1'),
     State(component_id='graph2', component_property='chart2'),
     State(component_id='table', component_property='data')]
)

def save(n1, n2, n3, n4, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, movie, highest_rate, star, dirctr, chart1, chart2, data):
    global top_10_profitable_movies, before_2000, after_2000, num_of_movies, highest_rated_movie, popular_actor, popular_director, df_all, fig, fig2
    
    ctx = dash.callback_context
    button = ctx.triggered[0]['prop_id'].split('.')[0]
    
    get_database()
    
    movie = num_of_movies
    highest_rate = highest_rated_movie
    star = popular_actor
    dirctr = popular_director 
    
    get_figure()

    chart1 = dcc.Graph(
                    id ='graph1',
                    figure = fig
             )
    chart2 = dcc.Graph(
                id='graph2',
                figure= fig2,
             )
    
    if n1==0:
        return [movie, highest_rate, star, dirctr, chart1, chart2, None]
    
    elif n1>0:

        if button == 'add-1':
            conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
            cur = conn.cursor()
            cur.execute('INSERT INTO imdb VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)', 
                        (v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13))
            cur.close()
            conn.commit()
            conn.close()

        elif button == 'delete-1':
            conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
            cur = conn.cursor()
            cur.execute("DELETE FROM imdb WHERE movie_id = %s", (v1,))
            cur.close()
            conn.commit()
            conn.close()

        elif button == 'update-1':
            conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
            cur = conn.cursor()
            cur.execute('UPDATE imdb SET title=%s, certificate=%s, duration=%s, genre=%s, rate=%s, metascore=%s,description=%s,  year=%s, directors=%s, stars=%s, votes=%s, gross=%s WHERE movie_id=%s', 
                        (v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v1))
            cur.close()
            conn.commit()
            conn.close()

        conn = psycopg2.connect("dbname=imdb user=postgres password=1234")
        sql = "select * from imdb;"
        dat = sqlio.read_sql_query(sql, conn)
        conn.close()
        

        data_table = dash_table.DataTable(
                    id='table_all',
                    data=dat.to_dict('records'),
                    columns=[{'id': c, 'name': c} for c in dat.columns],
                    style_cell={'textAlign': 'center', 'width': '100px', 'minWidth': '100px', 'maxWidth': '100px'},
                    fixed_rows={'headers': True, 'data': 0},
                    style_header={'fontWeight': 'bold'},
                    page_size=4,
                    selected_rows=[],
                    style_table={'overflowX': 'auto'},
                    editable=True )

        return [movie, highest_rate, star, dirctr, chart1, chart2, data_table]
                            
if __name__ == '__main__':
    app.run_server()  

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2021 03:37:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:37:52] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:37:52] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:37:52] "GET /_favicon.ico?v=1.20.0 HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:37:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:42:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:42:19] "GET /_dash-component-suites/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2021 03:42:19] "GET /_dash-component-suites/dash_table/async-table.js HTTP/1.1" 200 -
